In [16]:
import os
import json
import collections
import matplotlib.pyplot as plt
import numpy as np
import collections

In [17]:
cdir = %pwd
cdir

'/Users/keikagawa/Documents/GitHub/earthquake'

In [18]:
data_path = os.path.join(cdir, 'data/data_test.json')
with open(data_path) as f:
    data = json.load(f)
word_list_tmp = []
for i in range(len(data)):
    word_list_tmp.append(data[i]['word'].lower())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# SeleniumでJavaScriptを動かす

In [ ]:
from time import sleep                 # スリープを使うために必要
from selenium import webdriver         # Webブラウザを自動操作する（python -m pip install selenium)
import chromedriver_binary             # パスを通すためのコード
from selenium.webdriver.common.by import By

In [ ]:
# テストコード
driver = webdriver.Chrome()            # Chromeを準備
driver.get('https://www.google.com/')  # Googleを開く
time.sleep(5)                          # 5秒間待機
driver.quit()                          # ブラウザを閉じる

In [ ]:
# Googleニュースサイトを確認
browser = webdriver.Chrome() #ブラウザの立ち上がる
browser.get('https://news.google.com/topstories?tab=wn&hl=ja&gl=JP&ceid=JP:ja') #google newsを開く
top = browser.find_element_by_class_name('DY5T1d') #クラスでトップ記事の見出しを取得

top.text #タグの中身の文字を取得

<ipython-input-6-5ed283419a6f>:3: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  top = browser.find_element_by_class_name('DY5T1d') #クラスでトップ記事の見出しを取得


'東部の攻防 ゼレンスキー大統領 “部隊は踏みとどまっている”'

In [ ]:
# 気象庁のページを確認
# クリックされずに非表示になっていることを確認
url = 'https://www.data.jma.go.jp/svd/eqev/data/daily_map/20220430.html'
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)

In [ ]:
# クリックして表示されたものを保存
date = '20220618'
# date = '20230510'
url = f'https://www.data.jma.go.jp/svd/eqev/data/daily_map/{date}.html'
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
# driver.quit()

submenu = driver.find_element(by=By.CLASS_NAME, value='subMenu')
driver.execute_script("arguments[0].style.display='block';", submenu)
data_org = submenu.text

In [ ]:
# data
print(data_org)

年   月 日 時 分 秒    緯度       経度       深さ(km)  Ｍ   震央地名
-----------------------------------------------------------------------------------------
2022  6 18 00:00 21.6  45°20.0'N 141°39.0'E   34     0.2  宗谷地方北部                  
2022  6 18 00:02 19.6  24° 6.7'N 123°43.5'E   21     3.1  西表島付近                   
2022  6 18 00:04  4.2  37°50.8'N 141°47.3'E   41     1.0  福島県沖                    
2022  6 18 00:04 13.3  39° 5.0'N 142°10.6'E   41     0.8  岩手県沖                    
2022  6 18 00:06  5.2  32° 1.6'N 131°56.4'E   27     1.1  日向灘                     
2022  6 18 00:06  6.3  36°17.7'N 138° 4.3'E    4    -0.6  長野県中部                   
2022  6 18 00:07 54.1  39°42.3'N 141°52.3'E   46     0.0  岩手県沿岸北部                 
2022  6 18 00:08 55.4  34°50.7'N 137°52.7'E   18     1.4  静岡県西部                   
2022  6 18 00:09 10.1  38°48.4'N 141°44.7'E   85     0.7  宮城県沖                    
2022  6 18 00:14 26.4  35°54.1'N 138° 8.2'E   10    -0.6  長野県南部                   

2022  6 18 00:14 32.5  27°5

# 複数日時のデータ取得

In [5]:
from time import sleep                 # スリープを使うために必要
from selenium import webdriver         # Webブラウザを自動操作する（python -m pip install selenium)
import chromedriver_binary             # パスを通すためのコード
from selenium.webdriver.common.by import By

import pandas as pd
import re

In [19]:
# 関数化
def get_data_earthquake(date):
    # クリックして表示されたものを保存
    url = f'https://www.data.jma.go.jp/svd/eqev/data/daily_map/{date}.html'
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    # driver.quit()

    submenu = driver.find_element(by=By.CLASS_NAME, value='subMenu')
    driver.execute_script("arguments[0].style.display='block';", submenu)
    data_org = submenu.text

    data = data_org
    data = re.sub(r'---+', '', data_org)
    data = data.replace('\n\n', '\n')
    data = data.replace(r':', '\t')
    data = data.replace(r'° ','°')
    data = data.replace('°', '\t')
    data = data.replace("'N", '')
    data = data.replace("'E", '')
    data = data.replace('緯度', '緯度(度)\t緯度(分)')
    data = data.replace('経度', '経度(度)\t経度(分)')

    df_org = pd.DataFrame(np.array(data.split()).reshape(-1,13))
    df_column = df_org.iloc[0].to_list()
    df = df_org.drop(df_org.index[[0]])
    df.columns = df_column

    # 保存
    df.to_csv(f'./data/{date}.csv', index=False)

In [20]:
from datetime import datetime
from datetime import timedelta

# 日付条件の設定
strdt = datetime.strptime("20230401", '%Y%m%d')  # 開始日
enddt = datetime.strptime("20230510", '%Y%m%d')  # 終了日

# 日付差の日数を算出（リストに最終日も含めたいので、＋１しています）
days_num = (enddt - strdt).days + 1  # （参考）括弧の部分はtimedelta型のオブジェクトになります

datelist = []
for i in range(days_num):
    datelist.append((strdt + timedelta(days=i)).strftime("%Y%m%d"))

# 確認
for d in datelist:
    print(d)


20230401
20230402
20230403
20230404
20230405
20230406
20230407
20230408
20230409
20230410
20230411
20230412
20230413
20230414
20230415
20230416
20230417
20230418
20230419
20230420
20230421
20230422
20230423
20230424
20230425
20230426
20230427
20230428
20230429
20230430
20230501
20230502
20230503
20230504
20230505
20230506
20230507
20230508
20230509
20230510


In [21]:
for date in datelist:
    get_data_earthquake(date)
    sleep(10)

# データの整形

In [ ]:
import pandas as pd
import re


In [ ]:
data = data_org
data = re.sub(r'---+', '', data_org)
data = data.replace('\n\n', '\n')
data = data.replace(r':', '\t')
data = data.replace(r'° ','°')
data = data.replace('°', '\t')
data = data.replace("'N", '')
data = data.replace("'E", '')
data = data.replace('緯度', '緯度(度)\t緯度(分)')
data = data.replace('経度', '経度(度)\t経度(分)')
print(data)

年   月 日 時 分 秒    緯度(度)	緯度(分)       経度(度)	経度(分)       深さ(km)  Ｍ   震央地名
2022  4 30 00	00 31.7  27	24.6 128	50.7   31     1.6  沖縄本島近海                  
2022  4 30 00	03 53.0  44	39.7 144	21.7  267     2.5  網走沖                     
2022  4 30 00	04  1.1  34	44.7 133	0.2   18     0.5  広島県北部                   
2022  4 30 00	05 15.7  39	3.4 140	51.9   10     0.6  岩手県内陸南部                 
2022  4 30 00	05 35.4  36	9.0 141	54.2   12     1.5  茨城県沖                    
2022  4 30 00	06 26.1  37	50.2 141	36.4   48     1.3  福島県沖                    
2022  4 30 00	06 53.6  31	55.5 130	50.1    2     0.7  鹿児島県薩摩地方                
2022  4 30 00	11  2.3  31	51.0 132	8.6   23     1.6  日向灘                     
2022  4 30 00	12 12.1  31	36.5 130	12.6   11     0.7  薩摩半島西方沖                 
2022  4 30 00	14 22.7  43	48.8 147	5.1    0     2.9  北海道東方沖                  
2022  4 30 00	16 28.4  33	31.9 135	32.1    7     0.9  和歌山県南部                  
2022  4 30 00	18 10.8  38	54.1 142	2.6   42     1.4  宮城県沖         

In [ ]:
df_org = pd.DataFrame(np.array(data.split()).reshape(-1,13))
df_column = df_org.iloc[0].to_list()
df = df_org.drop(df_org.index[[0]])
df.columns = df_column
df

,年,月,日,時,分,秒,緯度(度),緯度(分),経度(度),経度(分),深さ(km),Ｍ,震央地名
1,2022,4,30,00,00,31.7,27,24.6,128,50.7,31,1.6,沖縄本島近海
2,2022,4,30,00,03,53.0,44,39.7,144,21.7,267,2.5,網走沖
3,2022,4,30,00,04,1.1,34,44.7,133,0.2,18,0.5,広島県北部
4,2022,4,30,00,05,15.7,39,3.4,140,51.9,10,0.6,岩手県内陸南部
5,2022,4,30,00,05,35.4,36,9.0,141,54.2,12,1.5,茨城県沖
...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,2022,4,30,23,47,53.8,33,32.6,135,25.3,29,0.7,和歌山県南方沖
699,2022,4,30,23,47,55.5,33,31.4,133,54.2,30,0.2,高知県東部
700,2022,4,30,23,48,7.7,35,21.2,133,51.4,5,-0.4,鳥取県中部
701,2022,4,30,23,49,4.4,33,34.3,135,25.8,27,0.3,和歌山県南方沖


In [ ]:
# 保存
df.to_csv('./data/220430.csv', index=False)

In [ ]:
df1 = pd.read_csv('./data/220430.csv')
df1

,年,月,日,時,分,秒,緯度(度),緯度(分),経度(度),経度(分),深さ(km),Ｍ,震央地名
0,2022,4,30,0,0,31.7,27,24.6,128,50.7,31,1.6,沖縄本島近海
1,2022,4,30,0,3,53.0,44,39.7,144,21.7,267,2.5,網走沖
2,2022,4,30,0,4,1.1,34,44.7,133,0.2,18,0.5,広島県北部
3,2022,4,30,0,5,15.7,39,3.4,140,51.9,10,0.6,岩手県内陸南部
4,2022,4,30,0,5,35.4,36,9.0,141,54.2,12,1.5,茨城県沖
...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,2022,4,30,23,47,53.8,33,32.6,135,25.3,29,0.7,和歌山県南方沖
698,2022,4,30,23,47,55.5,33,31.4,133,54.2,30,0.2,高知県東部
699,2022,4,30,23,48,7.7,35,21.2,133,51.4,5,-0.4,鳥取県中部
700,2022,4,30,23,49,4.4,33,34.3,135,25.8,27,0.3,和歌山県南方沖
